In [1]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from smolagents import CodeAgent,DuckDuckGoSearchTool, HfApiModel,load_tool, tool, TransformersModel, Model, FinalAnswerTool, LiteLLMModel, UserInputTool
from custom_model import CustomTransformersModel
from typing import Optional
import torch
import yaml
import requests
from json import loads, dumps

In [8]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join("..")))
from smolagents import CodeAgent, Model, LiteLLMModel
from src.aiagent.utils.ecom_tools import (search_on_amazon,
                                          ParserProductDescriptionWithGuideTool, GetProductDescriptionTool,
                                          CompareProductTool, FilterProduct, FinalAnswerTool)
from src.aiagent.ui.main_gradio import GradioUI
import yaml
from src.aiagent.core.custom_python_executor import LocalPythonExecutor

In [2]:
from utils_tools import (get_price_from_product_url, search_on_amazon,
                         ParserProductDescriptionWithGuideTool, GetProductDescriptionTool,
                         CompareProductTool, FilterProduct, FinalAnswerTool)

In [3]:
from litellm import completion

In [4]:
with open('secrets.yaml') as f:
    SECRETS = yaml.safe_load(f)
os.environ["HF_TOKEN"] = SECRETS['hf_token']
os.environ["GEMINI_API_KEY"] = SECRETS['gemini_token']

In [5]:
model = LiteLLMModel(model_id='gemini/gemini-2.0-flash')

In [6]:
final_answer = FinalAnswerTool()

In [6]:
compare_products = CompareProductTool(model)

In [20]:
tv_search_results = search_on_amazon(keyword="TV")
print(tv_search_results)

[{'product_name': 'RCA TV 32 Pouces (80 cm) sans Bord LED HD Téléviseurs avec Triple Tuner (DVB-T/T2-C-S/S2), HDMI, CI+, Lecteur Multimédia USB, Mode Hôtel, Dolby Audio (NRB32HQ1, 2024)', 'image_url': 'https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg', 'product_link': 'https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6', 'price': '119,99 €', 'delivery_date': 'Livraison GRATUITE ven. 23 maiOu livraison accélérée jeu. 22 mai'}, {'product_name': 'Smart Tech 24HN01V3 HD LED TV 24 Pouces (60cm) Triple Tuner Dolby Audio H.265

In [12]:
comparison_table = compare_products(tv_search_results)

In [13]:
comparison_table

,0
0,{'product_name': 'RCA TV 32 Pouces (80 cm) LED...
1,{'product_name': 'Smart Tech 24HN01V3 HD LED T...
2,{'product_name': '[App intégrée] Vidéoprojecte...
3,{'product_name': 'THOMSON 32'' HD LED Fire TV ...
4,{'product_name': 'TD Systems Smart TV 40 Pouce...


In [11]:
final_answer('blabla', comparison_table)

('blabla',
                                                    0
 0  {'product_name': 'RCA TV 32 Pouces (80 cm) LED...
 1  {'product_name': 'Smart Tech 24HN01V3 HD LED T...
 2  {'product_name': 'VidéoProjecteur Portable Min...
 3  {'product_name': 'THOMSON 32'' HD - LED Fire T...
 4  {'product_name': 'Samsung 55Q70D - TV QLED 55 ...)

In [27]:
product_description = tv_search_results[0]

In [28]:
product_description

{'product_name': 'RCA TV 32 Pouces (80 cm) sans Bord LED HD Téléviseurs avec Triple Tuner (DVB-T/T2-C-S/S2), HDMI, CI+, Lecteur Multimédia USB, Mode Hôtel, Dolby Audio (NRB32HQ1, 2024)',
 'image_url': 'https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg',
 'product_link': 'https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6',
 'price': '119,99 €',
 'delivery_date': 'Livraison GRATUITE ven. 23 maiOu livraison accélérée jeu. 22 mai'}

In [21]:
model = LiteLLMModel(model_id='gemini/gemini-2.0-flash')

In [29]:
messages = [{"role": "system",
                     "content": ("Tu es un super assistant très fort pour resumer et structurer des json."
                                 "Je vais te fournir un json et tu vas faire en sorte qu'aucune valeur de clef soit superieur à 50 characère."
                                 "Tu as le droit de resumer pour conserver que les informations principales mais tu nas pas le droit de toucher les champs qui sont des urls, tu les rends tel quel sans les affecter."
                                 "Ne change pas la structure du json, il ne doit pas manquer de clef qui étaient presentent initialement ni etre renommé"
                                 "concernant les champs de livraison, je veux que tu ne conserve que la date la plus courte de livraison sous le format jour mois")},
                    {"role": "user", "content": f"met moi en forme ce json stp : {product_description}"}]

In [33]:
res = model(messages, response_format={"type": "json"}).content

SyntaxError: invalid character '…' (U+2026) (3676893262.py, line 1)

In [31]:
product_description

{'product_name': 'RCA TV 32 Pouces (80 cm) sans Bord LED HD Téléviseurs avec Triple Tuner (DVB-T/T2-C-S/S2), HDMI, CI+, Lecteur Multimédia USB, Mode Hôtel, Dolby Audio (NRB32HQ1, 2024)',
 'image_url': 'https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg',
 'product_link': 'https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6',
 'price': '119,99 €',
 'delivery_date': 'Livraison GRATUITE ven. 23 maiOu livraison accélérée jeu. 22 mai'}

In [39]:
res.replace('json', '', 1)

'```\n{\n  "product_name": "RCA TV 32 Pouces LED HD Triple Tuner HDMI USB Dolby",\n  "image_url": "https://m.media-amazon.com/images/I/71RY2UdWBWL._AC_UL320_.jpg",\n  "product_link": "https://www.amazon.fr/RCA-T%C3%A9l%C3%A9viseurs-Lecteur-Multim%C3%A9dia-NRB32HQ1/dp/B0D31PQ1SH/ref=sr_1_6?dib=eyJ2IjoiMSJ9.QEUyDt9Z8vIYCGRwKsSgwkn4cvCBfn931_DqH29zif2VxWwo7tfgMS7IwZ7A2ur-OLS1eqh7MMvNADPGgyZ_uBsi73hZQKfonhNSQVs6qu2jAki0ceLboxT3DCzA0rhEUAPfjrVT78lhwRcSQoYC41Jis_NFkYiwi6nhYQ9sK6eAaw1CsQL2dhMRSTnt8JUzyQ6he4dOaO2kwBajO1SB2rieseSJYhb3WGDTT_bD0UBcP1R7nN3_dX0dS82Kt5-F-8i9cQK0ZixaipSPv5Fij7rXVE21N1kaI0qzL3xjw4M.lLMf8bTpAQ-ZXei6_oP4rIySUGfT5FKTFanLN9cjSbQ&dib_tag=se&keywords=TV&qid=1747694146&sr=8-6",\n  "price": "119,99 €",\n  "delivery_date": "22 mai"\n}\n```'

In [19]:
model = LiteLLMModel(model_id='gemini/gemini-2.0-flash')

with open('../config/secrets.yaml') as f:
    SECRETS = yaml.safe_load(f)

# os.environ['HF_HOME'] = '/home/ayoub/llm_models'
os.environ['GEMINI_API_KEY'] = SECRETS['gemini_token']

In [20]:
filter_product = FilterProduct(model)

In [9]:
import time                                                                                                                                                                           
from datetime import date, timedelta                                                                                                                                                  
                                                                                                                                                                                    
today = date.today()                                                                                                                                                                  
next_weekend_start = today + timedelta((6 - today.weekday()) % 7)  # Saturday                                                                                                         
next_weekend_end = next_weekend_start + timedelta(days=1)  # Sunday     

In [ ]:
today 

In [ ]:
timedelta((6 - today.weekday()) % 7)

In [ ]:
next_weekend_start

In [32]:
timedelta((6 - today.weekday()) % 7)

datetime.timedelta(days=2)

In [10]:
next_weekend_start

datetime.date(2025, 6, 1)

In [11]:
print(f"Today is {today}, next weekend is {next_weekend_start} - {next_weekend_end}")  

Today is 2025-05-30, next weekend is 2025-06-01 - 2025-06-02


In [12]:
delivery_before = next_weekend_start.strftime("%d %B")

In [13]:
delivery_before

'01 June'

In [14]:
wedding_dresses = search_on_amazon(keyword="black dress for wedding")   

In [15]:
wedding_dresses

[{'product_name': 'Anaya with Love',
  'image_url': 'https://m.media-amazon.com/images/I/61BhxrOZfBL._AC_UL320_.jpg',
  'product_link': 'https://www.amazon.fr/Anaya-Love-Shoulder-Sleeveless-Bridesmaid/dp/B0DCFXSCD4/ref=sr_1_6?dib=eyJ2IjoiMSJ9.a42FcJQfuAJjh9eO9WqL2iG0jEpU2KQatLUToZ8i8uMncCDukkbwb_WxYIB8FnP-733j-OeclesSgGt9X5TyGkFzjcVotvfELM9Z8g1PFXPEpXRfsYrFWBLRtBGUmZ9qETUwWk3A89Y_Xkrk44i4Omq4CnY4rL601RSPxZ-Hl3g0BLNygBRmmtDOWszDKjBtiX8utFGj7CxpWCk_L5XHV-A5EVeU0dTgJtZUH6rk0I5YOFPp0tVTjeApshYjpBQnP6YdId4d6MUWIEBI-Y_bnbtRFAtLb1AwlfoFnayCr9Q.PvDm-ZziCMHKn0Uge2ZThRngH4obnS8OKNq7JTm-BOA&dib_tag=se&keywords=black+dress+for+wedding&qid=1748609326&sr=8-6',
  'price': '69,99 €',
  'delivery_date': ' Livraison GRATUITE mar. 3 juin '},
 {'product_name': 'ShinyBeauty',
  'image_url': 'https://m.media-amazon.com/images/I/81k0tLRgQML._AC_UL320_.jpg',
  'product_link': 'https://www.amazon.fr/ShinyBeauty-tissu-paillettes-m%C3%A8tre-Argent/dp/B01LZQMMBP/ref=sr_1_7?dib=eyJ2IjoiMSJ9.a42FcJQfuAJjh9eO9WqL2iG

In [21]:
filter_condition = f"suitable for a wedding and must be delivered before {delivery_before}"                                                                                           
filtered_wedding_dresses = filter_product(wedding_dresses, filter_condition)    

In [23]:
filtered_wedding_dresses

[]

In [28]:
def check_condition(product, condition):

    messages = [
                {"role": "system", "content": (
                    "Tu es un assistant chargé d'évaluer si un produit respecte une condition utilisateur."
                    "Tu vas recevoir un produit sous forme de dictionnaire, et une condition."
                    "Tu dois répondre uniquement par 'oui' ou 'non', selon que le produit satisfait la condition ou non en t'aidant des differents champs du dictionnaire."
                    "Explique moi à chaque fois pourquoi ta réponse"
                )},
                {"role": "user", "content": f"Produit : {product}\n Condition : {condition}"}
            ]

    result = model(messages).content.strip().lower()

    return result

In [29]:
check_condition(wedding_dresses[0], filter_condition)

'non. la date de livraison (3 juin) est après le 1er juin.'

In [30]:
wedding_dresses[0]

{'product_name': 'Anaya with Love',
 'image_url': 'https://m.media-amazon.com/images/I/61BhxrOZfBL._AC_UL320_.jpg',
 'product_link': 'https://www.amazon.fr/Anaya-Love-Shoulder-Sleeveless-Bridesmaid/dp/B0DCFXSCD4/ref=sr_1_6?dib=eyJ2IjoiMSJ9.a42FcJQfuAJjh9eO9WqL2iG0jEpU2KQatLUToZ8i8uMncCDukkbwb_WxYIB8FnP-733j-OeclesSgGt9X5TyGkFzjcVotvfELM9Z8g1PFXPEpXRfsYrFWBLRtBGUmZ9qETUwWk3A89Y_Xkrk44i4Omq4CnY4rL601RSPxZ-Hl3g0BLNygBRmmtDOWszDKjBtiX8utFGj7CxpWCk_L5XHV-A5EVeU0dTgJtZUH6rk0I5YOFPp0tVTjeApshYjpBQnP6YdId4d6MUWIEBI-Y_bnbtRFAtLb1AwlfoFnayCr9Q.PvDm-ZziCMHKn0Uge2ZThRngH4obnS8OKNq7JTm-BOA&dib_tag=se&keywords=black+dress+for+wedding&qid=1748609326&sr=8-6',
 'price': '69,99 €',
 'delivery_date': ' Livraison GRATUITE mar. 3 juin '}

In [ ]:
filtered_products = []

for product in list_product_element:
    try:
        if self._check_condition_with_llm(product, condition):
            filtered_products.append(product)
    except Exception as e:
        # Log ou passer en silence
        continue
